<a href="https://colab.research.google.com/github/Only-Mike/ADHD/blob/main/Topic_Modelling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Import data and concat data

In [1]:
import pandas as pd
import numpy as np

In [2]:
!git clone 'https://github.com/Only-Mike/ADHD.git'

Cloning into 'ADHD'...
remote: Enumerating objects: 56, done.
remote: Counting objects: 100% (56/56), done.
remote: Compressing objects: 100% (52/52), done.
remote: Total 56 (delta 25), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (56/56), done.


In [4]:
data1 = pd.read_csv('/content/ADHD/datasets/adhd2018.csv')
data2 = pd.read_csv('/content/ADHD/datasets/adhd2019.csv')
data3 = pd.read_csv('/content/ADHD/datasets/adhd2020.csv')
data4 = pd.read_csv('/content/ADHD/datasets/adhd2021.csv')
data5 = pd.read_csv('/content/ADHD/datasets/adhd2022.csv')
frames = [data1, data2, data3, data4, data5] #creating frame for all datasets
df = pd.concat(frames) #Concat all datasets to "df"

#Prep for data handling/Topic Modelling

In [5]:
#Installing preprocessor to celan our text
!pip install tweet-preprocessor -q

# Installing Gensim and PyLDAvis
!pip install -qq -U gensim
!pip install -qq pyLDAvis

# explainability (why did the model say it's related to this author)
!pip install eli5

     |████████████████████████████████| 24.1 MB 1.7 MB/s 
     |████████████████████████████████| 1.7 MB 8.5 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 216 kB 7.7 MB/s 
     |████████████████████████████████| 133 kB 46.9 MB/s 
  Created wheel for eli5: filename=eli5-0.13.0-py2.py3-none-any.whl size=107748 sha256=4a462cf1db55c75661976063b3346602fb4a4cb0e349786dd5525201f27ce932
  Stored in directory: /root/.cache/pip/wheels/cc/3c/96/3ead31a8e6c20fc0f1a707fde2e05d49a80b1b4b30096573be
Successfully built eli5
  Attempting uninstall: jinja2
    Found existing installation: Jinja2 2.11.3
    Uninstalling Jinja2-2.11.3:
      Successfully uninstalled Jinja2-2.11.3
ERROR: pip's dependency resolver does not currently take into ac

In [6]:
import tqdm #progress bar
import preprocessor as prepro # text prepro

import spacy #spacy for quick language prepro
nlp = spacy.load('en_core_web_sm') #instantiating English module

# sampling, splitting
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import train_test_split


# loading ML libraries
from sklearn.pipeline import make_pipeline #pipeline creation
from sklearn.feature_extraction.text import TfidfVectorizer #transforms text to sparse matrix
from sklearn.linear_model import LogisticRegression #Logit model
from sklearn.metrics import classification_report #that's self explanatory
from sklearn.decomposition import TruncatedSVD #dimensionality reduction
from xgboost import XGBClassifier

import altair as alt #viz

#explainability
import eli5
from eli5.lime import TextExplainer

# topic modeling

from gensim.corpora.dictionary import Dictionary # Import the dictionary builder
from gensim.models import LdaMulticore # we'll use the faster multicore version of LDA

# Import pyLDAvis
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

%matplotlib inline
pyLDAvis.enable_notebook()

/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable
/usr/local/lib/python3.7/dist-packages/past/builtins/misc.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Mapping


In [7]:
# prepro settings
prepro.set_options(prepro.OPT.URL, prepro.OPT.NUMBER, prepro.OPT.RESERVED, prepro.OPT.MENTION, prepro.OPT.SMILEY)

# Topic modelling

In [8]:
df.head()

,Authors,Author(s) ID,Title,Year,Cited by,Link,Affiliations,Authors with affiliations,Abstract,Author Keywords,EID
0,"Danielson M.L., Bitsko R.H., Ghandour R.M., Ho...",57212872865;22133380600;6507545467;55931285700...,Prevalence of Parent-Reported ADHD Diagnosis a...,2018,470.0,https://www.scopus.com/inward/record.uri?eid=2...,National Center on Birth Defects and Developme...,"Danielson, M.L., National Center on Birth Defe...",The purpose of this study is to estimate the n...,NaN,2-s2.0-85043366583
1,"Savage J.E., Jansen P.R., Stringer S., Watanab...",56108103800;57194519231;54407564900;5720462432...,"Genome-wide association meta-analysis in 269,8...",2018,431.0,https://www.scopus.com/inward/record.uri?eid=2...,"Department of Complex Trait Genetics, Center f...","Savage, J.E., Department of Complex Trait Gene...",Intelligence is highly heritable 1 and a major...,NaN,2-s2.0-85048943479
2,"Cortese S., Adamo N., Del Giovane C., Mohr-Jen...",8651678100;36151905300;35572846200;55968380300...,Comparative efficacy and tolerability of medic...,2018,403.0,https://www.scopus.com/inward/record.uri?eid=2...,"Center for Innovation in Mental Health, Academ...","Cortese, S., Center for Innovation in Mental H...",Background: The benefits and safety of medicat...,NaN,2-s2.0-85051242636
3,"Sayal K., Prasad V., Daley D., Ford T., Coghil...",6701748666;7203041562;7006105495;7202760757;66...,"ADHD in children and young people: prevalence,...",2018,327.0,https://www.scopus.com/inward/record.uri?eid=2...,"Division of Psychiatry and Applied Psychology,...","Sayal, K., Division of Psychiatry and Applied ...",Attention-deficit hyperactivity disorder (ADHD...,NaN,2-s2.0-85030750499
4,"Franke B., Michelini G., Asherson P., Banasche...",57211894018;55959024400;35402700900;6603935963...,"Live fast, die young? A review on the developm...",2018,229.0,https://www.scopus.com/inward/record.uri?eid=2...,"Department of Human Genetics, Donders Institut...","Franke, B., Department of Human Genetics, Dond...",Attention-deficit/hyperactivity disorder (ADHD...,Adult-onset ADHD; Cognitive impairment; Comorb...,2-s2.0-85052888875


In [9]:
data = df[['Authors', 'Author(s) ID','Title', 'Abstract','Year', 'Cited by']]